In [20]:
import requests
import pandas as pd 
import json
import pymongo
from pymongo import MongoClient

In [21]:
# table_header = 'Total Private-Total Private-Average Weekly Earnings of All Employees, In Dollars'
table_name = 'AvgWeeklyEarningsAllEmp.csv'

In [23]:
df_master = pd.read_excel('../../master_list.xlsx', converters={'Primary_State_FIPS' : lambda x : str(x)})
df_master.head()

,CBSA,MSA_Name,Primary_City,Primary_State,Primary_State_Code,Primary_State_FIPS,Primary_County,Primary_County_FIPS,lat,lng,County_agg,County_FIPS_agg,zips
0,10180,"Abilene, TX",Abilene,Texas,TX,48,Taylor,48441,32.4543,-99.7384,"Callahan County, Jones County, Taylor County","48059, 48253, 48441",79605 79699 79603 79601 79607 79606 79602 7960...
1,10420,"Akron, OH",Akron,Ohio,OH,39,Summit,39153,41.0798,-81.5219,"Portage County, Summit County","39133, 39153",44306 44307 44304 44305 44302 44303 44301 4430...
2,10500,"Albany, GA",Albany,Georgia,GA,13,Dougherty,13095,31.5776,-84.1762,"Dougherty County, Lee County, Terrell County, ...","13095, 13177, 13273, 13321",31721 31701 31707 31705 31702 31703 31706 31708
3,10540,"Albany-Lebanon, OR",Albany,Oregon,OR,41,Linn,41043,44.6274,-123.0966,Linn County,41043,97321 97322 97355
4,10580,"Albany-Schenectady-Troy, NY",Albany,New York,NY,36,Albany,36001,42.6664,-73.7987,"Albany County, Rensselaer County, Saratoga Cou...","36001, 36083, 36091, 36093, 36095",12208 12209 12204 12206 12207 12202 12203 1220...


In [24]:
df_list = pd.read_csv('../temp.csv')
df_list

,CBSA,MSA_Name,area_code,Unnamed: 3,Unnamed: 4
0,10180,"Abilene, TX",10180,NaN,NaN
1,10420,"Akron, OH",10420,NaN,NaN
2,10500,"Albany, GA",10500,NaN,NaN
3,10540,"Albany-Lebanon, OR",10540,NaN,NaN
4,10580,"Albany-Schenectady-Troy, NY",10580,NaN,NaN
...,...,...,...,...,...
379,49420,"Yakima, WA",49420,NaN,NaN
380,49620,"York-Hanover, PA",49620,NaN,NaN
381,49660,"Youngstown-Warren-Boardman, OH-PA",49660,NaN,NaN
382,49700,"Yuba City, CA",49700,NaN,NaN


In [25]:
df_listmaker = df_master[['CBSA', 'Primary_State_FIPS']].copy()
df_listmaker.head()

,CBSA,Primary_State_FIPS
0,10180,48
1,10420,39
2,10500,13
3,10540,41
4,10580,36


In [26]:
df_listmaker = df_listmaker.merge(df_list, on='CBSA')
df_listmaker

,CBSA,Primary_State_FIPS,MSA_Name,area_code,Unnamed: 3,Unnamed: 4
0,10180,48,"Abilene, TX",10180,NaN,NaN
1,10420,39,"Akron, OH",10420,NaN,NaN
2,10500,13,"Albany, GA",10500,NaN,NaN
3,10540,41,"Albany-Lebanon, OR",10540,NaN,NaN
4,10580,36,"Albany-Schenectady-Troy, NY",10580,NaN,NaN
...,...,...,...,...,...,...
379,49420,53,"Yakima, WA",49420,NaN,NaN
380,49620,42,"York-Hanover, PA",49620,NaN,NaN
381,49660,39,"Youngstown-Warren-Boardman, OH-PA",49660,NaN,NaN
382,49700,06,"Yuba City, CA",49700,NaN,NaN


In [18]:
len(df_listmaker)

384

In [27]:
df = pd.DataFrame(columns=["area_code","series id","year","period","period_name","value"])
series = []
for index, row in df_listmaker.iterrows():
    # construct series id by concatenating: Prefix + State Code + CBSA Code + Supersector Code/Industry Code + Data Type Code
    series.append('SMU' + row['Primary_State_FIPS'] + str(row['area_code']) + '05000000' + '11')

In [29]:
series[6]

'SMU22107800500000011'

In [32]:
lister = [series[i:i + 50] for i in range(0, len(series), 50)]

for i in range(0, len(lister)):
    request_series(lister[i])
    print(i)

0
1
2
3
4
5
6
7


In [30]:
def request_series(series_list):
    #  print(series_list)
    headers = {'Content-type': 'application/json'}
    data = json.dumps({"seriesid": series_list,"startyear":"2010", "endyear":"2019","registrationkey":"<key here>"})
    p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
    json_data = json.loads(p.text)
    if json_data['status'] == 'REQUEST_SUCCEEDED':
        add_to_df(json_data)
    else:
        print(json_data['status'])  

In [31]:
def add_to_df(data):
    for series in data['Results']['series']:
        seriesId = series['seriesID']
        cbsaID = seriesId[5:10]
        for item in series['data']:
            year = item['year']
            period = item['period']
            period_name = item["periodName"]
            value = item['value']
            if 'M01' <= period <= 'M12':
                df.loc[len(df)] = [cbsaID,seriesId,year,period,period_name,value] 
 

In [33]:
df

,area_code,series id,year,period,period_name,value
0,10180,SMU48101800500000011,2019,M12,December,743.21
1,10180,SMU48101800500000011,2019,M11,November,703.10
2,10180,SMU48101800500000011,2019,M10,October,718.06
3,10180,SMU48101800500000011,2019,M09,September,740.00
4,10180,SMU48101800500000011,2019,M08,August,747.40
...,...,...,...,...,...,...
45319,49740,SMU04497400500000011,2010,M05,May,523.89
45320,49740,SMU04497400500000011,2010,M04,April,537.83
45321,49740,SMU04497400500000011,2010,M03,March,537.59
45322,49740,SMU04497400500000011,2010,M02,February,533.98


In [34]:
df.to_csv(table_name, index=False)

In [61]:
df = pd.read_csv('../CSVs/AvgWeeklyEarningsAllEmp.csv')
df

,area_code,series id,year,period,period_name,value
0,10180,SMU48101800500000011,2019,M12,December,743.21
1,10180,SMU48101800500000011,2019,M11,November,703.10
2,10180,SMU48101800500000011,2019,M10,October,718.06
3,10180,SMU48101800500000011,2019,M09,September,740.00
4,10180,SMU48101800500000011,2019,M08,August,747.40
...,...,...,...,...,...,...
45679,19340,SMU19269800500000011,2010,M05,May,542.23
45680,19340,SMU19269800500000011,2010,M04,April,545.10
45681,19340,SMU19269800500000011,2010,M03,March,540.63
45682,19340,SMU19269800500000011,2010,M02,February,547.15


In [60]:
ser_45540 = df[df.area_code == 45540]
ser_45540

,area_code,series id,year,period,period_name,value
40812,45540,SMU12455400500000011,2019,M12,December,832.12
40813,45540,SMU12455400500000011,2019,M11,November,809.36
40814,45540,SMU12455400500000011,2019,M10,October,816.20
40815,45540,SMU12455400500000011,2019,M09,September,803.85
40816,45540,SMU12455400500000011,2019,M08,August,785.40
...,...,...,...,...,...,...
40915,45540,SMU12455400500000011,2011,M05,May,731.28
40916,45540,SMU12455400500000011,2011,M04,April,761.20
40917,45540,SMU12455400500000011,2011,M03,March,706.92
40918,45540,SMU12455400500000011,2011,M02,February,695.36


In [50]:
df_temp = pd.DataFrame(df.groupby('area_code').count())
df_temp.reset_index(inplace=True)
df_temp.sort_values(by='value').head(50)

,area_code,series id,year,period,period_name,value
171,27980,108,108,108,108,108
84,19300,108,108,108,108,108
148,25940,108,108,108,108,108
352,48060,108,108,108,108,108
308,42700,108,108,108,108,108
133,24420,108,108,108,108,108
36,14010,108,108,108,108,108
348,47460,108,108,108,108,108
38,14100,108,108,108,108,108
312,43420,108,108,108,108,108


In [9]:
df_pain = pd.read_excel('../pain_in_my_ass.xlsx')
df_pain

,CBSA,MSA_Name,area_code
0,11500,"Anniston-Oxford, AL",11500
1,12220,"Auburn-Opelika, AL",12220
2,13820,"Birmingham-Hoover, AL",13820
3,19300,"Daphne-Fairhope-Foley, AL",19300
4,19460,"Decatur, AL",19460
...,...,...,...
379,39100,"Poughkeepsie-Newburgh-Middletown, NY",3600001
380,39150,"Prescott Valley-Prescott, AZ",39140
381,39300,"Providence-Warwick, RI-MA",77200
382,44140,"Springfield, MA",78100


In [10]:
df_pain.drop(columns='area_code', inplace=True)
df_pain = df_pain.merge(df_temp, on='CBSA', how='left')
df_pain

,CBSA,MSA_Name,series id,year,period,period_name,value
0,11500,"Anniston-Oxford, AL",120.0,120.0,120.0,120.0,120.0
1,12220,"Auburn-Opelika, AL",120.0,120.0,120.0,120.0,120.0
2,13820,"Birmingham-Hoover, AL",120.0,120.0,120.0,120.0,120.0
3,19300,"Daphne-Fairhope-Foley, AL",108.0,108.0,108.0,108.0,108.0
4,19460,"Decatur, AL",120.0,120.0,120.0,120.0,120.0
...,...,...,...,...,...,...,...
379,39100,"Poughkeepsie-Newburgh-Middletown, NY",NaN,NaN,NaN,NaN,NaN
380,39150,"Prescott Valley-Prescott, AZ",NaN,NaN,NaN,NaN,NaN
381,39300,"Providence-Warwick, RI-MA",NaN,NaN,NaN,NaN,NaN
382,44140,"Springfield, MA",NaN,NaN,NaN,NaN,NaN


In [11]:
df_pain.to_csv('temp.csv')

In [58]:
df.area_code.value_counts()

24500    120
47260    120
30980    120
31140    120
47580    120
        ... 
45540    108
19300    108
46300    108
24260    108
13220    108
Name: area_code, Length: 384, dtype: int64

In [40]:
len(df['series id'].unique())

381

In [42]:
df_unique = pd.DataFrame(df['series id'].unique())
df_unique

,0
0,SMU48101800500000011
1,SMU39104200500000011
2,SMU13105000500000011
3,SMU41105400500000011
4,SMU36105800500000011
...,...
376,SMU53494200500000011
377,SMU42496200500000011
378,SMU39496600500000011
379,SMU06497000500000011


In [43]:
df_unique.to_csv('temp2.csv')

In [62]:
# Create instance of MongoClient
client = MongoClient()
# Connection URI
client = MongoClient('<conn string>')
# Select database
db = client['MSA']
# create new collection
collection = db.AvgWeeklyEarningsAllEmp_raw
# turn dataframe into readable format for mongo
df_dict = df.to_dict(orient='records')
# write dataframe to unemployment_predicted_2024 collection
collection.insert_many(df_dict)